# Deep learning workshop - **Part 1**

Let's dive into manual implementation of Neural Network - Multi Layer Perceptron.
In this notebook you will:
- Implement Linear (also called Dense, Fully-Connected) layer as a Perceptron.
- Implement Activation functions to add non-linearity
- Allow your solution to stack multiple layers to form MLP network.
- Perform forward propagation through your network.

This (and later) template implementation is similar to Pytorch framework.

## Task 1a:

Define a dataset that we can use later for training.
Declare a simple perceptron (Linear layer) that inherits defined class Module - it is here, to help you store all network layers.

The single layer perceptron should have:
1. Weights and Biases for each perceptron
...

In [1]:
# Import
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from collections import OrderedDict

## Define our dataset
Let's make it simple, create a circle dataset where **class 1** is inside of given `radius` and **class 2** is outside given `radius`.

In [74]:
def dataset_circles(m=10, radius=0.7, noise=0.0):
    # Element values are in the interval <-1; 1>
    X = (np.random.rand(m, 2, 1) * 2.0) - 1.0

    # Element-wise multiplication with random noise
    N = (np.random.rand(m, 2, 1) - 0.5) * noise
    Xnoise = X + N

    # Compute the radius
    # Element-wise square
    XSquare = Xnoise ** 2

    # Sum over axis=1. We get a (m, 1) array.
    RSquare = np.sum(XSquare, axis=1, keepdims=True)
    R = np.sqrt(RSquare)

    # Y is 1, if radius `R` is greater than `radius`
    Y = (R > radius).astype(float)

    # Return X, Y
    return X, Y

def draw_dataset(x, y):
    if x.shape[1] == 2:
        fig = px.scatter(x=x[:,0,0], y=x[:,1,0], color=y[:,0,0], width=500, height=500, color_continuous_scale='Bluered')
    else:
        return
    fig.show()

In [76]:
X,Y = dataset_circles(m=100)
draw_dataset(X,Y)

### Module

All deep learning frameworks have usually one elementary building block.
In our project, we follow the structure of the pytorch, so the elementary building block is called **`Module`**.
Now, it is pretty simple, but it will get more complex and more useful...
You can see function `.backward` that will later contain the partial derivations of chain rule for backward pass and parameter optimization.

In [28]:
class Module:
    def __init__(self):
        self.modules = OrderedDict()

    def add_module(self, module, name:str):
        if hasattr(self, name) and name not in self.modules:
            raise KeyError("attribute '{}' already exists".format(name))
        elif '.' in name:
            raise KeyError("module name can't contain \".\"")
        elif name == '':
            raise KeyError("module name can't be empty string \"\"")
        self.modules[name] = module

    def forward(self, *args, **kwargs) -> np.ndarray:
        pass

    def backward(self, *args, **kwargs):
        pass

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)


## Linear Layer

The place, where the mathematical magic happens.
To speedup weights multiplication in perceptron it is better to use vectorisation.

The activation function and layer logic are separated for easier backward propagation (chain rule) and optimization.


In [83]:
#------------------------------------------------------------------------------
#   Linear class
#------------------------------------------------------------------------------
class Linear(Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.W = np.random.randn(out_features, in_features)
        self.b = np.zeros((out_features, 1))

    def forward(self, input: np.ndarray) -> np.ndarray:
        print(f'W{self.W.shape} x I{input.shape}')
        net = self.W @ input + self.b
        return net

    def backward(self, dNet):
        pass


## Activations

The definitions for Sigmoid, ReLU, LeakyReLU, and CELU activation functions with forward and backward pass.
Let's start with the forward pass. (for now, we can leave the backward pass on `pass`)

In [32]:
#------------------------------------------------------------------------------
#   SigmoidActivationFunction class
#------------------------------------------------------------------------------
class Sigmoid(Module):
    def __init__(self):
        super(Sigmoid, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        return 1.0 / (1.0 + np.exp(-input))

    def backward(self, dNet):
        pass

#------------------------------------------------------------------------------
#   RELUActivationFunction class
#------------------------------------------------------------------------------
class ReLU(Module):
    def __init__(self):
        super(ReLU, self).__init__()

    def forward(self, input: np.ndarray) -> np.ndarray:
        return np.maximum(input, 0)

    def backward(self, dNet):
        pass

#------------------------------------------------------------------------------
#   ContinuouslyDifferentiableExponentialLinearUnitActivation class
#------------------------------------------------------------------------------
class CELU(Module):
    def __init__(self, alpha=1.0):
        super(CELU, self).__init__()
        self.alpha=alpha

    def forward(self, input: np.ndarray) -> np.ndarray:
        return (np.maximum(0, input) + np.minimum(0, self.alpha * (np.exp(input/self.alpha)-1)))

    def backward(self, dNet):
        pass

#------------------------------------------------------------------------------
#   LeakyRELUActivationFunction class
#------------------------------------------------------------------------------
class LeakyReLU(Module):
    def __init__(self, slope=0.2):
        super(LeakyReLU, self).__init__()
        self.slope=slope

    def forward(self, input: np.ndarray) -> np.ndarray:
        return np.maximum(input, self.slope*input)

    def backward(self, dNet):
        pass

### Plotting the functions
Verify our implementations of Activation functions - do the graphs look like they should?

In [33]:
activationsInput = np.linspace(-4,4,100)

sigmoid = Sigmoid()
y = sigmoid.forward(activationsInput)

fig = make_subplots(rows=2, cols=2)

fig.add_trace(
    go.Scatter(x=activationsInput, y=y, name='Sigmoid'),
    row=1, col=1
)

tanh = CELU()
y = tanh.forward(activationsInput)
fig.add_trace(
    go.Scatter(x=activationsInput, y=y, name='CELU'),
    row=1, col=2
)

relu = ReLU()
y = relu(activationsInput)
fig.add_trace(
    go.Scatter(x=activationsInput, y=y, name='ReLU'),
    row=2, col=1
)

leakyrelu = LeakyReLU()
y = leakyrelu(activationsInput)
fig.add_trace(
    go.Scatter(x=activationsInput, y=y, name='LeakyReLU'),
    row=2, col=2
)

fig.update_layout(height=600, width=800, title_text="Activation functions")
fig.show()


### Perceptron feed forward

Model your Perceptron.
Define and initialize perceptron with "1 neuron"
Feed `X` to the perceptron and see the results.

In [71]:
X,Y = dataset_circles(m=4)

fc = Linear(X.shape[1],1)
print(fc.forward(X))

(1, 2) x (4, 2, 1)
[[[-1.94314071]]

 [[ 1.9192153 ]]

 [[ 2.34139837]]

 [[ 2.60169726]]]


Your Single Layer Perceptron with an Activation function
Use previously defined perceptron and use its output as input for the activation function.
Feed `X` to the perceptron again and see if something changes.

In [ ]:
ac = CELU()
net = fc.forward(X)
net = ac.forward(net)
print(net)


## Task 1b:

Let's implement the `Model` which should contain all of our `Modules` and the call for simple forward feed.

### Model class

Implementation of the **`Model`** class.
Define its forward function - the implementation of forward and backward pass is sensitive to the order of called operations.
Each Layer(module) of type **`Module`** can be saved to the attribute **`Module.modules`** using the **`add_module`** method.


In [78]:
#------------------------------------------------------------------------------
#   Model class
#------------------------------------------------------------------------------
class Model(Module):
    def __init__(self):
        super(Model, self).__init__()

    def forward(self, input):
        features = input
        for module in self.modules:
            features = self.modules[module].forward(features)
        return features

    def backward(self, dA: np.ndarray):
        pass

In [84]:
model = Model()
model.add_module(Linear(X.shape[1], 4), 'input')
model.add_module(ReLU(), 'relu1')
model.add_module(Linear(4,5), 'hidden1')
model.add_module(LeakyReLU(),'leakyrelu1')
model.add_module(Linear(5,1), 'output')
model.add_module(Sigmoid(), 'Sigm1')

In [85]:
print(model.forward(X))

W(4, 2) x I(100, 2, 1)
W(5, 4) x I(100, 4, 1)
W(1, 5) x I(100, 5, 1)
[[[0.50582509]]

 [[0.54313384]]

 [[0.45453376]]

 [[0.53887134]]

 [[0.4624218 ]]

 [[0.1430781 ]]

 [[0.61818975]]

 [[0.22422177]]

 [[0.18520475]]

 [[0.53835505]]

 [[0.56644571]]

 [[0.68136953]]

 [[0.5895116 ]]

 [[0.18383886]]

 [[0.59582685]]

 [[0.52383775]]

 [[0.45741873]]

 [[0.59118582]]

 [[0.56189718]]

 [[0.59892824]]

 [[0.33279679]]

 [[0.41041085]]

 [[0.51417211]]

 [[0.54781541]]

 [[0.36541932]]

 [[0.24374128]]

 [[0.25041755]]

 [[0.53317153]]

 [[0.30856992]]

 [[0.59656804]]

 [[0.59533125]]

 [[0.22863537]]

 [[0.50953284]]

 [[0.33656283]]

 [[0.2580435 ]]

 [[0.52750461]]

 [[0.52730914]]

 [[0.59191903]]

 [[0.26611799]]

 [[0.54751661]]

 [[0.14735784]]

 [[0.6099388 ]]

 [[0.41786757]]

 [[0.52930645]]

 [[0.54991883]]

 [[0.57159601]]

 [[0.55879252]]

 [[0.51896009]]

 [[0.48805148]]

 [[0.28897408]]

 [[0.50977469]]

 [[0.59235782]]

 [[0.53188459]]

 [[0.26559245]]

 [[0.53452235